# Scanner - Pre-market MOST ACTIVE

## Brief
We'll be using `reqScannerData` to fetch most active stocks by volume during pre-market session and filtering through the list to find "hot" candidates for the day.

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 4002, clientId=17)

<IB connected to 127.0.0.1:4002 clientId=17>

Error 162, reqId 3: Historical Market Data Service error message:API scanner subscription cancelled: 3
Error 162, reqId 4: Historical Market Data Service error message:API scanner subscription cancelled: 4
Error 162, reqId 5: Historical Market Data Service error message:API scanner subscription cancelled: 5
Error 162, reqId 6: Historical Market Data Service error message:API scanner subscription cancelled: 6
Error 162, reqId 7: Historical Market Data Service error message:API scanner subscription cancelled: 7
Error 162, reqId 9: Historical Market Data Service error message:API scanner subscription cancelled: 9


In [ ]:
sub = ScannerSubscription(instrument='STK', locationCode='STK.US.MAJOR', scanCode='MOST_ACTIVE')

sub.abovePrice = 5.0
sub.belowPrice = 50.0
sub.aboveVolume = 100000

scanData = ib.reqScannerData(sub)

# Get top 20 most active stocks during premarket.
top_20 = scanData[:20]
contracts = [sd.contractDetails.contract for sd in top_20]

In [ ]:
# Get real-time tickers data
tickers = ib.reqTickers(*contracts)
ib.sleep(2) # Wait for data

volume_data = []
for ticker, scan in zip(tickers, top_20):
    symbol = ticker.contract.symbol
    volume = ticker.volume if ticker.volume is not None else 0
    exchange = ticker.contract.exchange
    volume_data.append({
        'Rank': scan.rank,
        'Symbol': symbol,
        'Volume': volume,
        'Exchange': exchange
    })

# Create DataFrame for clean output
volume_df = pd.DataFrame(volume_data)
top_20_volume = volume_df.sort_values(by='Volume', ascending=False).head(20)

# Print in readable format
print("\nTop 20 Stocks by Premarket volume")
print(top_20_volume.to_string(index=False))